<div style="background-color:#262626; text-align: center;">
<img src="https://fiapfunctions.blob.core.windows.net/datasets/capa.png">
</div>

# Theory

In [ ]:
import skimage
import numpy as np
import pandas as pd
from skimage import io
import librosa, librosa.display
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

# Introdução ao processamento de sinais

---

Dentro das atividades gerais do entitulado Aprendizado de Máquina, podemos generalizar os seguintes passos:

Em um alto nível de abstração, qualquer problema envolvendo aprendizado de máquina pode ser dividido em três tipos de tarefas: 

1. Dados (ingestão/coleta dos dados, limpeza/tratamento e aprimoramento de recursos)
2. Treinamento (construção dos modelos de aprendizado de máquina usando os dados como recursos)
3. Avaliação (métricas de avaliação do modelo)

Os recursos, definidos como **"propriedades mensuráveis individuais ou características de um fenômeno que está sendo observado"**, são muito úteis porque ajudam a máquina a entender os dados e a classificá-los em categorias ou prever valores.

Tipos de dados diferentes usam técnicas de processamento diferentes. No campo do **Processamanto de Linguagem Natural**, podemos citar o **Word2vec**, que funciona muito bem para processar textos. Ele cria uma representação das palavras em vetores numéricos, onde a distância entre dois vetores de palavras determina a semelhança entre as mesmas. Se tentarmos aplicar o Word2vec em outro campo, os resultados provavelmente não farão sentido.

<div align="center" style="width: 100%;">
    <img src="imgs/word2vec-linear-relationships.svg" style="width: 1000px;">
</div>

No campo da **Visão Computacional**, vamos pensar na imagem como tipo de dados: ela parece uma coisa para o olho humano, mas uma máquina a vê de maneira diferente após ser transformada em recursos numéricos derivados dos valores de cada pixel, e ainda com a possibilidade de se usar diferentes filtros (dependendo da aplicação).

In [ ]:
digit = skimage.io.imread('imgs/image_423.jpg')
skimage.io.imshow(digit)
plt.show()

In [ ]:
pd.DataFrame(digit)

In [ ]:
def visualize_input(img, ax):
    ax.imshow(img, cmap='gray')
    width, height = img.shape
    thresh = img.max()/2.5
    for x in range(width):
        for y in range(height):
            ax.annotate(str(round(img[x][y],2)), xy=(y,x),
                        horizontalalignment='center',
                        verticalalignment='center',
                        color='white' if img[x][y]<thresh else 'black')

In [ ]:
normalized = digit/255
fig = plt.figure(figsize = (14,14)) 
ax = fig.add_subplot(111)
visualize_input(normalized, ax)

In [ ]:
digit.shape

## Por que estudar DSP

<div align="center" style="width: 100%;">
    <img src="imgs/datascience-conway-venn-diagram.jpg" style="width: 750px;">
    <h4 style="font-family: courier; font-size: .8em;">ref: https://www.oreilly.com/ideas/beyond-the-venn-diagram</h4>
</div>

In [ ]:
s = [ 67, 111, 110, 102, 105, 114, 109, 101,  32, 113, 117, 101,  32, 117, 109,
      32, 116, 101, 120, 116, 111,  32, 112, 111, 100, 101,  32, 115, 101, 114,
      32,  99, 111, 110, 115, 105, 100, 101, 114,  97, 100, 111,  32, 117, 109,
      32, 115, 105, 110,  97, 108,  32, 110, 111,  32, 116, 101, 109, 112, 111, 33]

print(len(s))

In [ ]:
texto = '' . join(chr(v) for v in s)
print(texto)

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(s)
plt.show()

## 1. Sinal Analógico *vs* Digital

### Como é o áudio analógico

É um sinal contínuo que representa uma onda de som. A figura que segue é um desenho um sinal analógico.

In [ ]:
amp = 5
t = np.arange(0.0, 1.0, 0.02)

yadd = amp * np.sin(5*2*t*np.pi) + amp * np.sin(2*2*t*np.pi)

plt.figure(figsize=(16,4))
plt.stem(yadd, use_line_collection=True)

#sns.despine()
plt.show()

In [ ]:
plt.figure(figsize=(24,14))
count = 1

y0 = np.zeros_like(yadd)

for i in range(10, 30):
    ax = plt.subplot(5,4,count)
    y0[i] = yadd[i]    
    plt.stem(y0, use_line_collection=True)
    plt.xlim((0,50))
    plt.ylim((-10,10))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_ticks([])
    ax.yaxis.set_label_position("right")
    plt.ylabel('+', fontsize=22)
    y0[i] = 0
    count +=1

plt.ylabel(' ...', fontsize=22, rotation='horizontal', horizontalalignment='left')
plt.show()

# Fundamentos do processamento de áudio digitail

Um arquivo de áudio é uma representação de um sinal sonoro.  O som por sua vez, é o resultado de uma onda mecânica que é transmitida por um meio de propagação, como o  ar  ou  a  água.   Sendo  um  pouco  mais  detalhista,  o  som  é  o  resultado  direto  de  umadeterminada vibração que provoca zonas de maior e menor compressão, originando umaonda. Com esta definição, podemos afirmar que o silêncio é a ausência de qualquer som.Ao  falar  dos  sons,  estamos  falando  de  um  sinal  analógico  e,  portanto,  se  faznecessário a conversão do mesmo para um sinal digital. Isto é necessário para este sinalpossa ser processado computacionalmente, e este é o domínio do processamento de sinais. O processamento de sinais nos auxilia no processo de transformação de um sinalanalógico contínuo em um sinal discreto, que por sua vez se torna um sinal digital.  Paragarantir que ambos os tipos de processamento produzam resultados satisfatórios, deve-se recorrer a uma fundamentação matemática que descreva como a informação carregadapelos sinais envolvidos é afetada em cada etapa do processo.

Duas dimensões da figura:

* **Tempo**: é a medida do período.
* **Amplitude**: é a medida de mudança de um sinal durante um período.

<div align="center" style="width: 100%;">
    <img src="imgs/sine_voltage.svg">    
</div>

In [ ]:
time = np.arange(0, 10, 0.1)
amplitude   = np.sin(time)
plt.figure(figsize=(16,4))
plt.plot(time, amplitude)
plt.title('Sine wave')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.grid(True, which='both')
plt.axhline(y=0, color='k')
plt.show()
plt.show()

## Propriedades do Som

Podemos definir que uma onda sonora é o movimento das partículas causado por umaperturbação que se propaga através de um meio.  Uma onda sonora também é uma ondalongitudinal que consiste na alternância regular de pressão variando conforme a densi-dade, o que efetivamente propaga as flutuações em um ciclo finito.  O deslocamento daspartículas altera a densidade, que gera mudanças de pressão produzindo novos desloca-mentos de partículas.Em  relação  a  velocidade  ($\large \vec{v}$),  essa  grandeza  é  baseada  no  meio  de  propagaçãoutilizado.  A velocidade das ondas sonoras tende a ser maior nos sólidos visto que estes elementos possuem maior proximidade de suas partículas.  Outro fator é a temperaturado meio de propagação.  A temperatura é uma grandeza que mede o nível de agitaçãodas  partículas  do  meio.   Quanto  maior  a  temperatura  do  meio,  maior  a  velocidade  depropagação do som. Podemos considerar três propriedades fundamentais do som:

* Altura
* Intensidade
* Timbre.

### Altura

A **altura** define o aspecto do áudio que comumente chamamos de grave e agudo. Essa classificação é relacionada ao número de vibrações por segundo em uma onda sonora. Os sons com maior comprimento de onda tendem a ser mais baixos, e são intitulados como graves. Já os sons agudos possuem um menor comprimento de onda, gerando uma maior frequência.

In [ ]:
def plot_wav_freq_amp(freq, amplitude):
    sampling_freq = 500
    phases = np.pi*np.array(1/7)
    time = np.arange(-1, 1 + 1/sampling_freq, 1/sampling_freq) 
    sine_wave = amplitude * np.sin(2*np.pi*freq*time + phases)
    return sine_wave

fa_3_5 = plot_wav_freq_amp(3,5)
fa_9_5 = plot_wav_freq_amp(9,5)

plt.figure(figsize=(14,12))
plt.subplot(211)
plt.plot(fa_3_5, linewidth=2)
plt.title("Som baixo - grave", fontsize=18)
plt.axhline(y=0, color='green')

plt.subplot(212)
plt.plot(fa_9_5, linewidth=2)
plt.title("Som alto - agudo", fontsize=18)
plt.axhline(y=0, color='green')

plt.show()

### Intensidade

A **intensidade** diz respeito a amplitude do sinal. Um sinal considerado forte possui uma maior amplitude. Os sons fortes transportam uma maior quantidade de energia em relação aos sons fracos. Uma onda sonora perde intensidade devido a resistência oferecida pelas partículas durante sua propagação.

O nível sonoro determina a intensidade de um som, e poodemos medir o nível sonoro em decibéis ($\large dB$), sendo que $\large 1 dB$ é o nível mais baixo detectado pela audição humana, e $\large 120 dB$ é o nível máximo suportado por nossa audição.

In [ ]:
fa_3_5 = plot_wav_freq_amp(3,5)
fa_3_15 = plot_wav_freq_amp(3,15)

plt.figure(figsize=(14,12))
plt.subplot(211)
plt.plot(fa_3_5, linewidth=2)
plt.title("Som fraco - baixa amplitude", fontsize=18)
plt.axhline(y=0, color='green')
plt.ylim(-20, 20)

plt.subplot(212)
plt.plot(fa_3_15, linewidth=2)
plt.title("Som forte - alta amplitude", fontsize=18)
plt.axhline(y=0, color='green')
plt.ylim(-20, 20)

#plt.savefig('amplitude.png')
plt.show()

### Timbre

Por último temos o **timbre**.  Esta propriedade do som que nos permite distinguiruma fonte sonora de outra, mesmo que estajam em uma mesma frequência. Otimbre de uma fonte sonora é representado por uma onda complexa, que é a soma de umaonda fundamental (som puro, ou simples) e sons harmônicos.  Cada fonte sonora produzuma onda sonora complexa diferente, isso nos permite identificar que o som produzidopor um violino é diferente de um som produzido por um oboé.

In [ ]:
!curl https://meriatdatasets.blob.core.windows.net/public/digital-audio-processing/piano-C4.wav -o audios/piano-C4.wav
!curl https://meriatdatasets.blob.core.windows.net/public/digital-audio-processing/violin-C4.wav -o audios/violin-C4.wav
!curl https://meriatdatasets.blob.core.windows.net/public/digital-audio-processing/trumpet-C4.wav -o audios/trumpet-C4.wav

In [ ]:
x_1, sr_1 = librosa.load('audios/piano-C4.wav')
r_1 = librosa.autocorrelate(x_1, max_size=5000)

x_2, sr_2 = librosa.load('audios/violin-C4.wav')
r_2 = librosa.autocorrelate(x_2, max_size=5000)

x_3, sr_3 = librosa.load('audios/trumpet-C4.wav')
r_3 = librosa.autocorrelate(x_3, max_size=5000)

In [ ]:
plt.figure(figsize=(16,12))
plt.subplot(311)
plt.plot(r_1[:600], linewidth=2)
plt.title("Piano, C4 (261 Hz)", fontsize=16)
plt.axis('off')

plt.subplot(312)
plt.plot(r_2[:600], linewidth=2)
plt.title("Violino, C4 (261 Hz)", fontsize=16)
plt.axis('off')

plt.subplot(313)
plt.plot(r_3[:600], linewidth=2)
plt.title("Trompete, C4 (261 Hz)", fontsize=16)
plt.axis('off')

#plt.savefig('timbre.png')
plt.show()

## Conceitos básicos

### Filtragem 

O primeiro passo para converter o singal de analógico para digital é filtrar as componentes de altas freqüências. A maior parte da energia da fala está entre **200Hz** e **300Hz** e **2700Hz** e **2800Hz**. Então, assume-se uma largura de banda de **3000Hz** para fala. Uma banda de **4000Hz**, porém, é implementada nos equipamentos para evitar `aliasing`. 

### Amostragem 

O segundo passo para converter um sinal analógico de voz para digital é amostrar a entrada filtrada a uma freqüência de amostragem constante. Isso é feito através de um processo chamado `modulação por amplitude de pulso` (**PAM**). O sinal analógico original é usado para modular a amplitude um trem de pulsos com uma freqüência constante. O trem de pulsos move-se a uma freqüência constante, chamada **freqüência de amostragem**. Para determinar a freqüência de amostragem, usa-se o **Teorema de Nyquist**. 

Este teorema diz que a freqüencia de amostragem deve ser o dobro da maior freqüência do sinal amostrado. Se este teorema for obedecido, o sinal poderá ser reconstruído através de um filtro passa-baixas.

### Voz digitalizada 

Depois de filtrar e amostrar, deve-se digitalizar as amostras. O processo de digitalização é chamado `PCM`. Uma técnica chamada quantização é usada para digitalizar as amostras. 

### Pulse Code Modulation - PCM  

`PCM` é um método de codificação definido no **ITU-T G.711**. PCM: Pulse-code modulation - modulação por código de pulso. O PCM é o formato padrão de CDs de áudio, transmitindo dois canais com uma taxa de amostragem de **44.100 Hz** (amostras por segundo) e uma profundidade de **16 bits** por amostra.

### Quantização

Quando as amostras chegam na fase de quantização, elas são associadas a um intervalo de quantização. Cada intervalo de quantização é associado a um valor na forma de um número binário. Os intervalos são igualmente espaçados, assim tem-se a quantização uniforme. O tamanho padrão do número binário é **8bits**. Para um sinal amostrado a **8kHz**, a taxa de transmissão é de **64kbps**.

Quando uma amostra não é associada ao seu intervalo de quantização correto, é gerado ruído de quantização. Este ruído de quantização é equivalente ao ruído que interfere na `relação sinal ruído` (**SNR**) do sinal de voz. Um modo de reduzir o ruído de quantização é aumentar o número de intervalos de quantização, mas para isso deve-se aumentar o tamanho do valor de **8bits** para mais bits.

`SNR` é o fator que mais afeta a qualidade de voz na quantização uniforme. Para melhorar a qualidade de voz, a quantização uniforme é substituída pelo processo de quantização não-uniforme, chamado companding.

### A-law and u-law Companding  

`A-law` e `u-law` são algoritmos de compressão definidos pelo Consultative Committee for International Telephony And Telegraphy (CCITT) G.711 que comprime **PCM linear** de **16bits** em **8bits** logarítmico. 

### Sample Rate – Taxa de amostragem 

A interface de áudio é capaz de "fotografar" o sinal elétrico várias vezes durante um determinado período de tempo. A esta ação de “fotografar” chamamos de Amostragem. 

A frequência com que essa ação ocorre é chamada de Frequência de Amostragem (ou **Taxa de Amostragem**, ou **Sample Rate**), calculada por segundo, e é dada em `Hz` (hertz). Quanto maior a frequência de amostragem, mais informação é capturada do sinal elétrico a cada segundo, e mais fiel ao sinal original é o sinal digital. 

Quanto mais informação capturada, maior o espaço requerido para armazenar esta informação, e maior o poder de processamento processar estes dados. 

Portanto, não adianta você adquirir uma interface de áudio capaz de trabalhar com um sample rate de **96 kHz** (ou **96.000 Hz**), que irá gravar **96 mil pontos de informação** do seu sinal elétrico a cada segundo, se o seu computador possui pouca memória e um processador obsoleto. 

De acordo com o **Teorema de Nyquist**, o Sample Rate ideal deve ser sempre o dobro da frequência que se deseja registrar. Como o ouvido humano é capaz de perceber até **20 kHz**, uma frequência de amostragem de **40.000 Hz** é a ideal que você possa capturar toda a informação necessária para uma ótima qualidade de áudio. Trabalhar com **44.1 kHz** é mais do que suficiente. 

### Bit Rate (ou Bit Depth) – A Resolução da amostra 

A amostra é convertida para valores de **0** e **1** (binário). Quanto maior a quantidade de valores representando determinada amostra, maior é a resolução (bit rate) desta amostra. Resolução é a quantidade de informação binária (valores 0 e 1) de determinada amostra. 

> Quanto maior o bit rate, mais informação da amostra é armazenada, e mais fiel ao áudio original é o seu áudio digital. 

Uma amostra representada por um único **bit** é capaz de receber dois valores (0 ou 1). A fórmula para calcular a quantidade de valores em cada **bit** é $\large Q = 2^n$ (quantidade = dois elevado a $\large n$, sendo $\large n$ o número de bits da amostra).  

Para que dados multimídia sejam enviados, são usadas técnicas de processamento digital de sinais para codificá-los. 

### Formatos e tamanhos de arquivos

WAV é um formato de contêiner de arquivo de áudio da **Microsof/IBM**. É o formato principal usado nos sistemas Windows para armazenar áudio sem compressão e sem perdas. `WAV` é uma variante do método de formato de fluxo de bits `RIFF` para armazenar dados em "pedaços".

Esse padrão de "CD de áudio" consome aproximadamente 10 MB de espaço em disco para cada minuto de áudio. Em comparação, alguns formatos modernos, como o `FLAC`, que são compactados mas ainda sem perdas, levam cerca de **4 MB** de espaço em disco por minuto. Um formato compactado com perdas como o `MP3` ocupa muito menos espaço. Um `MP3` exportado do Audacity ou FFmpeg, com a taxa de bits padrão de **128 kbps** usa cerca de **1 MB** de espaço por minuto para um arquivo mono ou estéreo.

## Análise/Processamento de sinais 

### Sinal: definição e classificações iniciais

Em relação aos sinais, podem-se destacar as seguintes definições e considerações iniciais:

* Sinal: entidade que carrega informação.
* Visão matemática de sinal: variável funcionalmente dependente de uma ou mais variáveis independentes. Ex.: $\large y = f (x), z = f (x, y), w = f (x, y, z)$.
* Visão fı́sica de sinal: grandeza fı́sica.
* Tipos de sinais de acordo com o número de variáveis independentes:
    * Unidimensional. Ex.: áudio = $\large f (t)$.
    * Bidimensional. Ex.: imagem = $\large f (x, y)$.
    * Tridimensional. Ex.: vı́deo = $\large f (x, y, t)$.
    * Multidimensional. Ex.: tomografia/sismologia = $\large f (v 1 (t), v 2 (t), · · · , v V (t), t)$.
* Tipos de sinais de acordo com o tipo das variáveis:
    * Sinal analógico: todas as variáveis são contı́nuas.
    * Sinal amostrado: discretização das variáveis independentes (amostragem).
    * Sinal quantizado: discretização da variável dependente (quantização).
    * Sinal digital: todas as variáveis são discretas (amostragem + quantização).
* Sinal amostrado e sinal digital: conjunto ordenado de valores numéricos (seqüência numérica).

### Processamento de sinais

Algumas das definições básicas em processamento de sinais são as seguintes:

* Objeto do processamento: sinal (definido como uma entidade que carrega informação).
* Agente do processamento: sistema.
    * “Um sistema é um cojunto de elementos, que interagem entre si, com o objetivo de realizar uma determinada função”.
    * Arquitetura de um sistema: variáveis, elementos, topologia e função.
* Domı́nio do processamento: domı́nio no qual a função do agente é definida.
    * Tempo/espaço (forma) × freqüência (composição espectral).
* Ação do processamento: função exercida pelo agente sobre o objeto.
    * Conformação (tempo/espaço) × Alteração espectral (freqüência).
* Arquitetura genérica do processamento:
    * Sinal de entrada (ou estı́mulo ou excitação ou perturbação).
    * Condições iniciais ou estado inicial.
    * Sistema.
    * Sinal de saı́da (ou resposta).
* Nomenclatura usual: "Sinal" (sinal desejado) × "Ruı́do" (sinal indesejado).

#### Conceitos importantes

* A ideia do Processamento Discreto de Sinais Analógicos baseia-se em transformar o sinal analógico em um sinal discreto, processar o sinal discreto e transformar o sinal discreto processado em um sinal analógico.
* A ideia do Processamento Digital de Sinais Analógicos baseia-se em transformar o sinal analógico em um sinal discreto, transformar o sinal discreto em um sinal digital (números), processar o sinal digital, transformar o sinal digital processado em um sinal discreto e transformar o sinal discreto processado em um sinal analógico.
* Para garantir que ambos os tipos de processamento produzam resultados satisfatórios, deve-se recorrer a uma fundamentação matemática que descreva como a informação carregada pelos sinais envolvidos é afetada em cada etapa do processo.
* A modelagem de sinais e sistemas analógicos fundamenta-se na matemática contı́nua.
* A modelagem de sinais e sistemas discretos/digitais baseia-se na matemática discreta.
* Estes dois tipos de modelagem coexistem, independentemente um do outro.
* Porém, é possı́vel que se estabeleça uma relação entre eles. Com isso, pode-se representar um deles por meio do outro, pode-se teorizar um deles por meio do outro e pode-se realizar cálculos de um deles por meio do outro.
* Em alguns casos, essa representação mútua é uma relação biunı́voca exata. Em outros, isso não é possı́vel, obtendo-se apenas uma aproximação inexata de um modelo pelo outro.
* Os mecanismos que permitem a conexão entre os modelos analógico e discreto/digital são a amostragem e a interpolação.
* A amostragem permite a conexão “Analógico → Discreto” e, conseqüentemente, a conexão “Analógico → Digital” ou A/D.
* A interpolação possibilita a conexão “Discreto → Analógico” e, conseqüentemente, a conexão “Digital → Analógico” ou D/A.


## Sinal Contínuo x Discreto 

Os sinais analógicos são por natureza contínuos no tempo. São teoricamente constituídos por uma sequência de infinitos pontos. Quando digitalizamos esses sinais, eles passam por um processo conhecido por amostragem e são transformados numa sequência finita de pontos discretos. Essa transformação é realizada por componentes conhecidos como conversores **Analógicos / Digitais** ou simplesmente conversores A/D. A transformação dos sinais digitais em analógicos é realizada por componentes eletrônicos conhecidos por conversores Digitais / Analógicos, ou conversores D/A. Na FIGURA 3, pode-se observar um exemplo do processo de transformação do sinal analógico contínuo em um sinal discretizado.  

<div align="center" style="width: 100%;">
    <img src="imgs/amostragem.jpg">
</div>

O efeito decorrente devido à amostragem do sinal é visível: O sinal discretizado não é exatamente igual ao sinal analógico. Na **FIGURA ACIMA** o sinal discretizado é uma reprodução grosseira do sinal original. Isso tem uma série de consequências e implicações. Por outro lado, transformar um sinal contínuo em sinal discreto traz as vantagens de se poder adequar o  tamanho do sinal a ser processado à capacidade limitada de memória dos processadores. A memória do processador, por maior que seja, é sempre finita, assim como sua capacidade de processamento. Aqui já apontamos alguns dos problemas que devem ser levados em consideração e resolvidos quando se pensa em projetar sistemas de processamento digital de sinais.

## Sinais no domı́nio do tempo

Nesse capı́tulo são apresentados conceitos básicos relativos aos sinais manipulados pelos sistemas de processamento digital, que são as seqüências. Inicialmente, a associação de ı́ndice com tempo e as notações mais comuns para seqüências são apresentadas. Em seguida, as sequências são classificadas. Posteriormente, as operações básicas sobre sequências e as sequências mais comuns são exemplificadas. Finalmente, algumas relações de dependência entre sequências são evidenciadas.

### Conversão Analógico Digital de Sinais

Processar sinais de áudio, originalmente analógicos, através dos métodos e ferramentas oferecidos pela tecnologia digital requer que os sinais sejam convertidos de um domínio para o outro. É importante investigar se neste processo de conversão existem perdas ou corrupção da informação, e se existirem, tentar detectar a natureza e quantificar a deterioração sofrida pelo sinal.

A conversão de um sinal do domínio analógico para o digital, e vice-versa, é realizada pelos conversores A/D e D/A, que são circuitos eletrônicos que se apresentam sob diversas topologias, cada qual com suas vantagens e limitações. As características que diferenciam estes conversores entre si são a resolução e a faixa de freqüências com que são capazes de trabalhar.

Este capítulo trata de uma solução específica para a conversão A/D e D/A: os conversores sigma delta ($\normalsize \Sigma \Delta$). Sua principal característica é a utilização de circuitos cuja resolução pode ser muito menor do que a resolução do conversor como um todo. Como é apresentado no decorrer do texto, é possível obter uma resolução equivalente a 16 bits na conversão A/D e D/A utilizando um quantizador de apenas 1 bit (comparador), uma solução que encontra larga aplicação hoje em dia em sistemas de gravação e reprodução de áudio (e.g.: CD players).

Outra possível aplicação é a amplificação em potência de sinais de áudio. No caso da modulação sigma delta, a seqüência de bits resultante pode ser diretamente aplicada a um amplificador classe D e entregue com potência suficiente a um alto falante.

São apresentados e discutidos neste capítulo os principais aspectos da conversão A/D, como a amostragem e a quantização. A modulação sigma delta é caracterizada e comparada com a modulação por código de pulsos (PCM), que é a mais tradicional e utiliza quantizadores de alta resolução.

<div align="center" style="width: 100%;">
    <img src="imgs/figure-4-1.png">
</div>

Uma análise sobre as não linearidades do processo de quantização é desenvolvida, o que leva a uma razoável compreensão sobre o desempenho dos conversores A/D no que diz respeito à resolução.

### Conversão A/D e D/A

#### Amostragem e quantização

Na conversão analógico digital, ou digitalização, de um sinal elétrico, duas operações básicas são efetuadas sobre o sinal: a amostragem no tempo e a quantização na amplitude.

O processo de amostragem para um sinal analógico $x(t)$ é descrito pela Equação 4.1 e epresentado graficamente na Figura 4.1.

$\Large x[n] = x( n ⋅ T_s )$ , onde $\normalsize T_s$ é o período de amostragem e $\normalsize x[n]$ é o valor do sinal amostrado no instante $\normalsize n$.

No domínio da freqüência, a amostragem cria versões periodicamente repetidas do espectro do sinal amostrado, centradas em freqüências múltiplas da freqüência de amostragem $\normalsize f_s = 1/T_s$. Este comportamento é descrito pela Equação 4.2 e mostrado na Figura 4.2.

$\Large X_{s}\left( f\right) =\dfrac {1}{T_{s}} \sum ^{\infty }_{k=-\infty }X\left( f-kf_{s}\right)$ , onde $\normalsize X_{s}\left( f\right)$ representa o espectro do sinal amostrado.

Como é possível observar na Figura 4.2, se a largura da faixa espectral f B do sinal amostrado for maior que a metade da freqüência de amostragem f s , ocorrerá uma superposição do conteúdo espectral entre duas repetições adjacentes.

<div align="center" style="width: 100%;">
    <img src="imgs/figure-4-2.png">
</div>

Este fenômeno é chamado aliasing e é prevenido aplicando o sinal a ser amostrado num filtro passa baixas cuja freqüência de corte é menor que a metade da freqüência de amostragem. Na Figura 4.2 a resposta deste filtro antialiasing aparece em linhas tracejadas. É importante mencionar, embora seja óbvio, que trata-se de um filtro exclusivamente analógico, uma vez que precede o processo de digitalização do sinal. Assim sendo, sua implementação sofre das limitações práticas intrínsecas aos filtros analógicos, como por exemplo, a dificuldade em se conseguir um filtro de alta ordem (6a ou maior), o que é necessário quando deseja-se uma faixa de corte relativamente estreita, além dos problemas gerados com a variação brusca da fase na região de transição.

> O processo de amostragem é uma operação reversível, uma vez que nenhuma informação é perdida e o sinal pode ser perfeitamente reconstruído. A quantização na amplitude de um sinal é um processo não reversível, uma vez que mapea um número infinito (contínuo) de valores de entrada num número finito (discreto) de valores de saída.

Estas saídas são representadas normalmente como uma palavra digital com um número finito de algarismos binários (bits). Um quantizador com $\normalsize Q$ níveis de saída é usualmente caracterizado como tendo $\normalsize N$ bits de resolução, onde $\normalsize N=log_2 (Q)$. Assim, $\normalsize Q$ deve ser escolhido como sendo uma potência de 2, para que $\normalsize N$ seja um número inteiro. Se $V$ é a amplitude máxima do sinal de saída, somente valores de entrada separados de no mínimo $\normalsize \Delta=2V/(Q-1)$ podem ser distinguidos entre dois níveis diferentes, conforme apresenta a Figura 4.3.

Na Figura 4.1 é mostrado um exemplo de um sinal senoidal quantizado com 16 (2 4 ) níveis. Uma palavra digital de 4 bits é suficiente para representar todos os possíveis níveis de amplitude deste sinal. Na Figura 4.3 são mostradas as funções de transferência de dois tipos de quantizadores uniformes.

<div align="center" style="width: 100%;">
    <img src="imgs/figure-4-3.png">
</div>

O processo de quantização é inerentemente não linear, o que dificulta a sua análise. Além disso, o erro introduzido no processo depende da diferença entre a amplitude da entrada e o nível referido na saída. Este erro pode ser interpretado como um ruído adicionado no sinal, e o desempenho de um conversor A/D é mensurado justamente pela quantidade deste ruído introduzido. Conclui-se que quanto maior for o número de níveis de quantização, melhor será o desempenho do conversor.

In [ ]:
def plot(x, y, xlabel='', ylabel=''):
    plt.plot(x,y)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    sns.despine()
    ax = plt.gca()
    fig = plt.gcf()
    return ax, fig
freq = 2
y = amp * np.sin(freq*2*t*np.pi)
freq2 = 5
y2 = amp * np.sin(freq2*2*t*np.pi)

In [ ]:
plt.figure(figsize=(18,4))

yadd = y+y2
ysamp = yadd[::5]
tsamp = t[::5]
plt.plot(t, yadd)
plt.plot(t, yadd, 'o')
ax = plt.gca()
fig = plt.gcf()

# x grid
ax.set_xticks(t,)
ax.get_yaxis().set_ticks([])
ax.grid(b=True, which='major', linewidth=0.5, axis='x')
ax.set_xticklabels('')
plt.xlabel('Sampling')
plt.show()

In [ ]:
plt.figure(figsize=(18,4))

yadd = y+y2
ysamp = yadd[::5]
tsamp = t[::5]
plt.plot(t, yadd)
plt.plot(t, yadd, 'o')
ax = plt.gca()
fig = plt.gcf()

# y grid
ax.get_xaxis().set_ticks([])
ax.set_yticks(np.linspace(-12,12,24), minor=True)
ax.grid(b=True, which='major', linewidth=0.5, axis='y')
ax.grid(b=True, which='minor', linewidth=0.5, axis='y')
plt.ylabel('Quantization')

plt.show()

In [ ]:
def requantiza(sinal, nbits, B):
    if B >= nbits: # nada a fazer
        y = sinal
        return y

    M = 2**(nbits-1)
    Q = 2**(B-1)
    
    if max(sinal) - min(sinal) < 2: # sinal normalizado
        sinal2 = sinal * M
        if max(sinal2) > M:
            print('Checar sinal')            
            #print('sinal normalizado')
            return None;

    sinal2 = np.floor(sinal2);
    y = np.floor(sinal2/M*Q);
    y = y/Q;

    return y

In [ ]:
p = np.arange(-1,1,.1)
n = np.arange(0,len(p))

y0 = requantiza(p, 32, 1)
y1 = requantiza(p, 32, 2)
y2 = requantiza(p, 32, 3)
y3 = requantiza(p, 32, 4)
y4 = requantiza(p, 32, 8)

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(n, p, color='r', label='original')
plt.stem(n, y0, label='quantizado', use_line_collection=True)
plt.xlabel('n', fontsize=16)
plt.title('Quantização em 1 bit', fontsize=18)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(n, p, color='r', label='original')
plt.stem(n, y1, label='quantizado', use_line_collection=True)
plt.xlabel('n', fontsize=16)
plt.title('Quantização em 2 bit', fontsize=18)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(n, p, color='r', label='original')
plt.stem(n, y2, label='quantizado', use_line_collection=True)
plt.xlabel('n', fontsize=16)
plt.title('Quantização em 3 bit', fontsize=18)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(n, p, color='r', label='original')
plt.stem(n, y3, label='quantizado', use_line_collection=True)
plt.xlabel('n', fontsize=16)
plt.title('Quantização em 4 bit', fontsize=18)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(n, p, color='r', label='original')
plt.stem(n, y4, label='quantizado', use_line_collection=True)
plt.xlabel('n', fontsize=16)
plt.title('Quantização em 8 bit', fontsize=18)
plt.grid()
plt.show()

### Como é o áudio digital PCM

Um PCM é uma representação do sinal analógico que amostra regularmente a amplitude do sinal analógico quantizado em intervalos uniformes de tempo.

### **Profundidade em bits**

Número de níveis diferentes em que uma amostra pode ser quantizada.

<div align="center" style="width: 100%;">
    <img src="imgs/Figure-5.14-Wave-quantized-at-different-bit-depths.png" style="width: 800px;">
</div>

### **Taxa de amostragem**

É o número de amostras por intervalo de tempo geralmente medida em amostras por segundo.

<div align="center" style="width: 100%;">
    <img src="imgs/teces_164.gif">    
</div>

### Qualidade do áudio

Ou seja, quanto maior a profundidade em bits e maior a taxa de amostragem, mais próximo do sinal analógico.

<div align="center" style="width: 100%;">
    <img src="imgs/Vol36_Sep08_ThinkingDigitally-graph1.png" style="width: 600px;">    
</div>

Importância de se garantir a frequência de amostragem correta. Abaixo ilustra-se uma senoide de 95 Hz amostrada a 100 Hz.

In [ ]:
fs=100 # freq. amostragem, Hz
fa=95  # freq. sinal analógico, Hz
T=0.2  # duracao em segundos
t=np.arange(T*fs+1)/fs # tempo discreto
ta=np.arange(T*fs*100)/(fs*100) # tempo
pi = np.pi
sd=np.cos(2*pi*(fs-fa)*t)  # sinal discreto
sa=np.cos(2*pi*fa*ta)      # sinal analógico

plt.figure(figsize=(18,18))
plt.subplot(311)
plt.plot(ta,sa,color='k'), plt.grid()
plt.title("Sinal analógico, senoide de %g Hz" % fa, fontsize=18)
plt.xlabel("t, s", fontsize=14)
plt.subplot(312)
plt.stem(t,sd, use_line_collection=True)

plt.plot(ta,sa,color='k')
plt.plot(t,sd, ':')

plt.xlabel("t, s", fontsize=14)
plt.title("Senoide de %g Hz amostrada a %g Hz" % (fa, fs), fontsize=18)
plt.subplot(313)
plt.stem(np.arange(T*fs+1),sd, use_line_collection=True)
plt.title("Sinal discreto", fontsize=18)
plt.xlabel("n", fontsize=14)
plt.show()

In [ ]:
def nearest_point(data, points):
    data = data.copy()
    for i in range(len(data)):
        point = data[i]
        min_idx = abs(points - point).argmin()
        data[i] = points[min_idx]
    return data

print('4 bit resolution:', 20./16) 
print('8 bit resolution:', 20./256)
print('16 bit resolution:', 20./65536)

t = np.arange(0.0, 1.0, 0.001)
print(t.shape, yadd.shape)

In [ ]:
fig = plt.figure(figsize=(15,10))

yadd = amp * np.sin(2*2*t*np.pi)  + amp * np.sin(5*2*t*np.pi)

levels4 = np.linspace(-20, 20, 16)
levels8 = np.linspace(-20, 20, 32)
levels16 = np.linspace(-20, 20, 256)

y4 = nearest_point(yadd, levels4)
plt.subplot(2,3,1)
plt.step(t, y4)
plt.xlim((0,1))
plt.ylim((-10,10))
plt.ylabel('Sampling (x)', fontsize=24)

y8 = nearest_point(yadd, levels8)
plt.subplot(2,3,2)
plt.step(t, y8)
plt.xlim((0,1))
plt.ylim((-10,10))

y16 = nearest_point(yadd, levels16)
plt.subplot(2,3,3)
plt.step(t, y16)
plt.xlim((0,1))
plt.ylim((-10,10))

t = np.arange(0.0, 1.0, 0.1)

ylowsamp = amp * np.sin(2*2*t*np.pi)  + amp * np.sin(5*2*t*np.pi)
plt.subplot(2,3,4)
plt.step(t, ylowsamp)
plt.xlim((0,1))
plt.ylim((-10,10))
plt.ylabel('Quantization (y)', fontsize=24)
plt.xlabel('Low', fontsize=24)

t = np.arange(0.0, 1.0, 0.01)

ylowsamp = amp * np.sin(2*2*t*np.pi)  + amp * np.sin(5*2*t*np.pi)
plt.subplot(2,3,5)
plt.step(t, ylowsamp)
plt.xlim((0,1))
plt.ylim((-10,10))
plt.xlabel('Mid', fontsize=24)

t = np.arange(0.0, 1.0, 0.001)

ylowsamp = amp * np.sin(2*2*t*np.pi)  + amp * np.sin(5*2*t*np.pi)
plt.subplot(2,3,6)
plt.step(t, ylowsamp)
plt.xlim((0,1))
plt.ylim((-10,10))
plt.xlabel('High', fontsize=24)

plt.show()